           CÀI ĐẶT MÔ HÌNH

In [1]:
import import_ipynb
import GRU
import emotion
import Cleanes_data
from keras.models import load_model
AI_model = load_model('Movie_AI.keras')
memotion_data = GRU.emotion_data
movie_data= Cleanes_data.movie_data




importing Jupyter notebook from GRU.ipynb
importing Jupyter notebook from emotion.ipynb


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Shape of TF-IDF matrix: (8, 60)
[7 0 4 6 2 3 1 5]
Epoch 1/10


C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.0000e+00 - loss: 2.0998
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2614 - loss: 2.0889      
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2892 - loss: 1.9831     
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7979 - loss: 1.8308 
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5349 - loss: 1.8877     
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6841 - loss: 1.8181     
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 1.7341 
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 1.6186 
Epoch 9/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 1.5712 
Epoch 10/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 1.4542 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step
[[0.10608406 0.1332826  0.12649104 0.10451192 0.13018541 0.14012273
  0.15935922 0.09996298]
 [0.10585815 0.14557917 0.12

<string>:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

NameError: name 'json_str_to_dict' is not defined

                       XÂY DỰNG CHATBOT

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
def predict_emotion(user_response):
    processed_response = emotion.clean_text(user_response)

# Tạo một đối tượng TF-IDF
    text= TfidfVectorizer(max_features=1000)  # Số lượng từ vựng tối đa: 1000

# Fit và chuyển đổi dữ liệu văn bản thành ma trận TF-IDF
    matrix = text.fit_transform(processed_response)
    predicted_label = AI_model.predict(matrix)
    return predicted_label


In [ ]:
def chatbot():
    # Chờ người dùng nhập câu trả lời
    user_response = input("Ngày hôm nay bạn cảm thấy thế nào? ")

    # Dự đoán cảm xúc từ câu trả lời của người dùng
    predicted_emotion = predict_emotion(user_response)
    
    # Dựa vào cảm xúc dự đoán, đưa ra gợi ý phim tương ứng
    recommended_movies = get_recommended_movies(predicted_emotion)
    
    # Hiển thị gợi ý phim cho người dùng
    print("Dựa vào cảm xúc của bạn, một số gợi ý phim có thể phù hợp là:")
    for movie in recommended_movies:
        print(movie)

# Hàm để lấy danh sách phim gợi ý dựa trên cảm xúc dự đoán
def get_recommended_movies(predicted_emotion):
    # Dựa vào nhãn cảm xúc dự đoán, trả về danh sách phim tương ứng
    # Ví dụ: có thể sử dụng một bộ dữ liệu phim có sẵn và lọc các phim dựa trên cảm xúc
    recommended_movies =
    # ...
    return recommended_movies

# Chạy chatbot
chatbot()